In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt
import csv





### Define parameters

In [ ]:
n_subjects = 10
n_episodes_per_subject = 10
sequence_length = 240 # Desired length after alignment

In [ ]:
'''
data_nn = np.loadtxt('Autoencoder_Output_Correct.csv', delimiter=',')

Data_NN_inc = np.loadtxt('Autoencoder_Output_Incorrect.csv', delimiter=',')
'''

In [ ]:

# Data loading functions (assuming consistent file naming)
def load_angles(subject, episode, correct=True):
  prefix = "Segmented Movements/Vicon/Angles/" if correct else "Incorrect Segmented Movements/Vicon/Angles/"
  filename = f"{prefix}m01_s{subject:02d}_e{episode:02d}_angles.txt"
  return csv.reader(open(filename))

# Load correct and incorrect angle data
correct_angles = {}
incorrect_angles = {}
for subject in range(1, n_subjects + 1):
  for episode in range(1, n_episodes_per_subject + 1):
    correct_angles[(subject, episode)] = next(load_angles(subject, episode))
    incorrect_angles[(subject, episode)] = next(load_angles(subject, episode, correct=False))

In [ ]:
# Reorganize data into initial arrays
correct_sequences = []
incorrect_sequences = []
for subject in range(1, n_subjects + 1):
  for episode in range(1, n_episodes_per_subject + 1):
    correct_sequences.append(list(map(float, correct_angles[(subject, episode)])))
    incorrect_sequences.append(list(map(float, incorrect_angles[(subject, episode)])))

# Remove inconsistent episodes (adjust indices as needed)
correct_sequences = correct_sequences[1::] + correct_sequences[11::] + correct_sequences[21::] + correct_sequences[31::] + correct_sequences[41::] + correct_sequences[50::] + correct_sequences[61:63] + correct_sequences[64::] + correct_sequences[71::] + correct_sequences[81::] + correct_sequences[85::] + correct_sequences[90::]
incorrect_sequences = incorrect_sequences[1::] + incorrect_sequences[11::] + incorrect_sequences[21::] + incorrect_sequences[31::] + incorrect_sequences[41::] + incorrect_sequences[50::] + incorrect_sequences[61:63] + incorrect_sequences[64::] + incorrect_sequences[71::] + incorrect_sequences[81::] + incorrect_sequences[85::] + incorrect_sequences[90::]

In [ ]:
# Perform linear alignment
n_sequences = len(correct_sequences)
correct_aligned = []
incorrect_aligned = []
for sequence in correct_sequences:
  sequence_length_original = len(sequence)
  f = interp1d([1, sequence_length_original], sequence, kind='linear')
  correct_aligned.append(f(list(range(1, sequence_length + 1))))

for sequence in incorrect_sequences:
  sequence_length_original = len(sequence)
  f = interp1d([1, sequence_length_original], sequence, kind='linear')
  incorrect_aligned.append(f(list(range(1, sequence_length + 1))))

In [ ]:
# Transform data into matrices
correct_data = np.array(correct_aligned).T
incorrect_data = np.array(incorrect_aligned).T

In [ ]:
# Re-center and scale the data
data_mean_correct = np.mean(correct_data, axis=1, keepdims=True)
centered_correct_data = correct_data - data_mean_correct
scaling_value_correct = np.max(np.abs(centered_correct_data))

data_mean_incorrect = np.mean(incorrect_data, axis=1, keepdims=True)
centered_incorrect_data = incorrect_data - data_mean_incorrect
scaling_value_incorrect = np.max(np.abs(centered_incorrect_data))

data_correct = centered_correct_data / scaling_value_correct
data_incorrect = centered_incorrect_data / scaling_value_incorrect


In [ ]:
# Save the data (replace with your preferred saving method)
np.savetxt('Data_Correct.csv', data_correct, delimiter=',')
np.savetxt('Data_Incorrectrect_Y, delimiter=',')